In [1]:
import getpass
api_key = getpass.getpass("Enter your GROQ API key: ")

In [2]:
import dspy
from langchain_groq import ChatGroq
from langchain_core.messages import HumanMessage, SystemMessage

class GroqLM(dspy.LM):
    def __init__(self, model="llama-3.3-70b-versatile", **kwargs):
        super().__init__(model=model, **kwargs)
        self.model_name = model
        self.client = ChatGroq(model=model,
                               api_key=api_key,)

    def __call__(self, prompt=None, messages=None, **kwargs):
        # Handle both prompt and messages format
        if messages:
            # Convert DSPy messages to LangChain format
            lc_messages = []
            for msg in messages:
                if msg.get("role") == "system":
                    lc_messages.append(SystemMessage(content=msg["content"]))
                elif msg.get("role") == "user":
                    lc_messages.append(HumanMessage(content=msg["content"]))
            response = self.client.invoke(lc_messages)
        else:
            # Handle simple prompt format
            response = self.client.invoke([HumanMessage(content=prompt)])
        
        return [response.content]


/home/kelvin/miniconda3/envs/genai-dev/lib/python3.14/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# Configure DSPy to use the GroqLM
lm = GroqLM()
dspy.configure(lm=lm)

predict = dspy.Predict("question -> answer")
out = predict(question="indian army vs chinese army ")
print(out.answer)


The Indian Army and the Chinese Army are two of the largest and most powerful military forces in the world. Here's a comparison of the two:

**Personnel:**
- Indian Army: approximately 1.4 million active personnel
- Chinese Army (People's Liberation Army): approximately 2.2 million active personnel

**Tanks:**
- Indian Army: around 4,065 tanks (including Arjun, T-90, and T-72)
- Chinese Army: around 7,150 tanks (including Type 99, Type 96, and Type 90)

**Aircraft:**
- Indian Army: around 700 aircraft (including helicopters and transport planes)
- Chinese Army: around 2,755 aircraft (including helicopters, transport planes, and fighter jets)

**Naval Power:**
- Indian Navy: around 295 ships (including submarines, destroyers, and aircraft carriers)
- Chinese Navy: around 710 ships (including submarines, destroyers, and aircraft carriers)

**Nuclear Capabilities:**
- India: estimated 150-200 nuclear warheads
- China: estimated 290-320 nuclear warheads

**Defense Budget:**
- India: around

MODEL INIT WITH DSPy

In [ ]:
import dspy
lm = dspy.LM("gemini/gemini-2.5-flash", api_key=GEMINI_API_KEY)
dspy.configure(lm=lm)

1 . MATH 

In [19]:
math = dspy.ChainOfThought("question -> answer: float")
math(question="india vs pakistan who will win?")

AuthenticationError: litellm.AuthenticationError: AuthenticationError: OpenAIException - The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

In [7]:
lm.inspect_history(1)

In [8]:
class QA(dspy.Signature):
    """to given question generate answers in few words"""
    question = dspy.InputField(desc="User's Question")
    answer = dspy.OutputField(desc="often in few words")

predict = dspy.Predict(QA) #Instead of dspy.predict("question --> answer")
prediction = predict(question="who hit the final run in cirket wc 2011 ")
print(prediction)

Prediction(
    answer='Dhoni hit the run'
)


In [9]:
lm.inspect_history(1)

Chain of thought

In [10]:
gen_ans = dspy.ChainOfThought(QA)
gen_ans(question="who hit the final run in cirket wc 2011 ")

Prediction(
    reasoning='The 2011 ICC Cricket World Cup was won by India, who defeated Sri Lanka in the final. The final match was played on April 2, 2011, at the Wankhede Stadium in Mumbai, India. To determine who hit the final run, we need to recall the details of that match. India was chasing a target of 275 runs set by Sri Lanka and needed 4 runs off the last 3 balls when Dhoni and Yuvraj Singh were at the crease, but then Dhoni hit a six and finished the game.',
    answer='Dhoni'
)

Modules

In [11]:
# why modules
multi_step_question = "what is the capital of france? who is the president of france? "
gen_ans = dspy.ChainOfThought(QA)
gen_ans(question=multi_step_question)

Prediction(
    reasoning='To answer this question, we need to provide the capital of France and the current president of France. The capital of France is a well-known fact, and the president can be determined by looking up the current political leaders.',
    answer='Paris, Emmanuel Macron'
)

In [12]:
class DoubleChainOfThought(dspy.Module):
    """Module to perform double chain of thought reasoning"""
    
    def __init__(self):
        super().__init__()
        self.first_thought = dspy.ChainOfThought("question -> answer")
        self.second_thought = dspy.ChainOfThought("question -> answer")

    def forward(self, input_question):
        # First chain of thought
        intermediate_answer = self.first_thought(question=input_question)

        # Second chain of thought using the intermediate answer
        final_answer = self.second_thought(question=intermediate_answer.answer)

        return final_answer
double_cot = DoubleChainOfThought()
double_cot(input_question=multi_step_question)


Prediction(
    reasoning='The question provides information about the capital of France, which is Paris, and the current president of France, who is Emmanuel Macron as of the last update. However, it does not ask a specific question that requires a direct answer. Given the context, it seems the task is to confirm or restate the information provided about France.',
    answer='The capital of France is indeed Paris, and as of the last update, the President of France is Emmanuel Macron.'
)

Outputting Typed Predictors

In [13]:
from pydantic import BaseModel, Field
class AnswerConfidence(BaseModel):
    answer: str = Field(..., description="The generated answer")
    confidence: float = Field(..., description="Confidence score of the answer between 0 and 1")
class QAWithConfidence(dspy.Signature):
    """to given question generate answers with confidence score"""
    question = dspy.InputField(desc="User's Question")
    answer = AnswerConfidence = dspy.OutputField(desc="Answer with confidence score")

generate_with_confidence = dspy.ChainOfThought(QAWithConfidence)
generate_with_confidence(question = "who was the key factor in 2011 cirket worldcup")

Prediction(
    reasoning="The 2011 Cricket World Cup was won by India, and the key factor in their victory can be attributed to several players. However, one player who stood out and is often credited with playing a crucial role in India's success is Yuvraj Singh. He was the player of the tournament, scoring 362 runs and taking 15 wickets. His all-round performance was instrumental in India's victory, as he scored crucial runs and took important wickets throughout the tournament.",
    answer='Yuvraj Singh was the key factor in the 2011 Cricket World Cup, with a confidence score of 0.9.',
    AnswerConfidence='The answer is Yuvraj Singh, with a confidence score of 0.9, indicating a high level of certainty in the response.'
)

Classification

In [11]:
from typing import Literal

class Classification(dspy.Signature):
    """Classify the sentiment of a given text"""
    text: str = dspy.InputField(desc="Input text to classify")
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField(desc="Sentiment classification")
    confidence: str = dspy.OutputField()
classify = dspy.Predict(Classification)
result = classify(text="I love programming!")
print(result)

Prediction(
    sentiment='positive',
    confidence='high'
)


In [12]:
from typing import Literal

class Classify(dspy.Signature):
    """Classify sentiment of a given sentence."""

    sentence: str = dspy.InputField()
    sentiment: Literal["positive", "negative", "neutral"] = dspy.OutputField()
    confidence: float = dspy.OutputField()

classify = dspy.Predict(Classify)
classify(sentence="This book was super fun to read, though not the last chapter.")

Prediction(
    sentiment='Positive',
    confidence='Medium'
)

Information Extraction

In [13]:
class ExtractInfo(dspy.Signature):
    """Extract structured information from text."""

    text: str = dspy.InputField()
    title: str = dspy.OutputField()
    headings: list[str] = dspy.OutputField()
    entities: list[dict[str, str]] = dspy.OutputField(desc="a list of entities and their metadata")

module = dspy.Predict(ExtractInfo)

text = "Apple Inc. announced its latest iPhone 14 today." \
    "The CEO, Tim Cook, highlighted its new features in a press release."
response = module(text=text)

print(response.title)
print(response.headings)
print(response.entities)

Apple Announces iPhone 14
Product Announcement
- {"name": "Apple Inc.", "type": "Organization", "metadata": {}}
- {"name": "iPhone 14", "type": "Product", "metadata": {"manufacturer": "Apple Inc."}}
- {"name": "Tim Cook", "type": "Person", "metadata": {"title": "CEO", "organization": "Apple Inc."}}


RAG - Retrieval Argmented generation

In [19]:
colbertv2_wiki = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")
dspy.configure(rm=colbertv2_wiki, lm=lm)

In [ ]:
retriver = dspy.Retrieve(k=3)
topKpassages = retriver("Iphone")
print(topKpassages)

Agents

In [ ]:
def evaluate_math(expression: str):
    return dspy.PythonInterpreter({ }).execute(expression)


react = dspy.ReAct("question -> answer: float", tools=[evaluate_math])

pred = react(question="What is 9362158 divided by the year of birth of David Gregory of Kinnairdy castle?")
print(pred.answer)

5635.917455947665


Multi stage pipeline

In [16]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""

    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(desc="mapping from section headings to subheadings")

class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""

    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")

class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)

    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        
        # Handle case where section_subheadings might be a string or dict
        if isinstance(outline.section_subheadings, str):
            import json
            try:
                section_subheadings_dict = json.loads(outline.section_subheadings)
            except json.JSONDecodeError:
                # If parsing fails, create a simple structure
                section_subheadings_dict = {outline.sections[i] if isinstance(outline.sections, list) else "Section": [] for i in range(len(outline.sections) if isinstance(outline.sections, list) else 1)}
        else:
            section_subheadings_dict = outline.section_subheadings
        
        for heading, subheadings in section_subheadings_dict.items():
            section, subheadings = f"## {heading}", [f"### {subheading}" for subheading in subheadings]
            section = self.draft_section(topic=outline.title, section_heading=section, section_subheadings=subheadings)
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)

draft_article = DraftArticle()
article = draft_article(topic="World Cup 2002")

In [17]:
article

Prediction(
    title='2002 FIFA World Cup Overview: Teams, Matches, and Legacy',
    sections=['## Section\nThe 2002 FIFA World Cup was the 17th edition of the FIFA World Cup, held in South Korea and Japan from May 31 to June 30, 2002. A total of 32 teams participated in the tournament, with Brazil winning the championship by defeating Germany 2-0 in the final. The 2002 FIFA World Cup is notable for being the first World Cup to be held in Asia, and it featured a number of memorable matches and moments, including the surprise elimination of defending champions France in the group stage. The legacy of the 2002 FIFA World Cup continues to be felt, with the tournament serving as a catalyst for the growth of football in Asia and leaving a lasting impact on the sport.']
)

In [23]:
%pip install -U datasets

Note: you may need to restart the kernel to use updated packages.
